In [1]:
import random
from collections import Counter

DECK_COMPLETE = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']

#hit on 8 or less
#stay on 17 or more
#line determined by player's hand score, starting on a 9
#row determined by dealer's hand score, starting on a 2
#H for hit, D for double, S for stand
BASIC_STRATEGY = [['H','D','D','D','D','H','H','H','H','H'],
                  ['D','D','D','D','D','D','D','D','H','H'],
                  ['D','D','D','D','D','D','D','D','D','H'],
                  ['H','H','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H']]

In [2]:
def rand_card(deck_func):
    card = random.choice(deck_func)
    deck_func.remove(card)
    return card


def score(cards):
    score = 0
    possible_hands = []
    aces = 0

    for card in cards:
        if type(card) == int:
            score += card
        elif card in ["J", "Q", "K"]:
            score += 10
        else:
            score += 1
            aces += 1

    for i in range(aces + 1):
        if (score + i * 10) <= 21:
            possible_hands.append(score + i * 10)

    if (possible_hands) == []:
        return -1
    return max(possible_hands)

In [3]:
def bj_round(hit_on_17=False):

    deck_current = DECK_COMPLETE.copy()

    player_cards = [
        rand_card(deck_func=deck_current),
        rand_card(deck_func=deck_current),
    ]
    player_score = score(player_cards)
    dealer_cards = [rand_card(deck_func=deck_current)]
    dealer_score = score(dealer_cards)

    if player_score == 21:
        return ("Win", 21)

    game_state = "Player_Phase"
    strategy = ""

    while game_state == "Player_Phase":
        if player_score <= 8:
            strategy = "H"
        elif player_score >= 17 + hit_on_17:
            strategy = "S"
        else:
            strategy = BASIC_STRATEGY[(player_score - 9)][(dealer_score - 2)]

        if strategy == "D":
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if player_score == -1:
                return ("Loss", player_score)
            else:
                game_state = "Dealer_Phase"
        elif strategy == "H":
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if player_score == -1:
                return ("Loss", player_score)
        elif strategy == "S":
            game_state = "Dealer_Phase"

    while dealer_score < 17 + hit_on_17:
        dealer_cards.append(rand_card(deck_func=deck_current))
        dealer_score = score(dealer_cards)
        if dealer_score == -1:
            return ("Win", player_score)

    if player_score > dealer_score:
        return ("Win", player_score)
    elif player_score == dealer_score:
        return ("Draw", player_score)
    elif player_score < dealer_score:
        return ("Loss", player_score)

In [4]:
bj_round()

('Win', 15)

In [5]:
results = {"Win": [], "Draw": [], "Loss": []}

for _ in range(1000000):
    wdl, total = bj_round()
    results[wdl].append(total)

for key, values in results.items():
    print(f"{key}:")

    total = dict(Counter(values))

    sorted_dict = dict(sorted(total.items(), reverse=True))

    for key, value in sorted_dict.items():
        print(f"{key}:{round(value*100/sum(sorted_dict.values()), 3)}%")
    print("")

Win:
21:21.935%
20:25.168%
19:14.809%
18:11.491%
17:7.883%
16:3.669%
15:3.945%
14:3.97%
13:4.279%
12:2.768%
11:0.083%

Draw:
21:7.913%
20:33.056%
19:18.179%
18:19.261%
17:21.592%

Loss:
20:4.04%
19:7.539%
18:11.029%
17:15.445%
16:5.32%
15:5.632%
14:5.751%
13:5.972%
12:3.782%
11:0.103%
-1:35.388%



In [6]:
def bj_round_till_win_single(cum_loss=0, penalty_per_loss=2):
    current_round = bj_round()
    if current_round[0] == "Win":
        return (current_round[0], current_round[1] - cum_loss)
    if current_round[0] == "Draw":
        return bj_round_till_win_single(cum_loss, penalty_per_loss)
    if current_round[0] == "Loss":
        return bj_round_till_win_single(cum_loss + penalty_per_loss, penalty_per_loss)


bj_round_till_win_single()

('Win', 16)

In [7]:
# min score 7
for _ in range(4):
    player = []
    for _ in range(6):
        player.append(max(bj_round_till_win_single()[1], 7))
    print(sorted(player, reverse=True))

[20, 18, 18, 16, 11, 8]
[21, 21, 21, 14, 11, 10]
[19, 18, 18, 12, 12, 10]
[20, 18, 18, 16, 14, 14]


In [8]:
def bj_round_till_win_group(size=6, cum_loss=0, penalty_per_loss=1):
    group = []
    while len(group) < size:
        current_round = bj_round()
        if current_round[0] == "Win":
            group.append(current_round[1] - cum_loss)
        if current_round[0] == "Draw":
            cum_loss += penalty_per_loss
        if current_round[0] == "Loss":
            cum_loss += penalty_per_loss

    return group


bj_round_till_win_group()

[19, 16, 17, 14, 9, 9]

In [9]:
for _ in range(4):
    print(bj_round_till_win_group())

[14, 18, 18, 8, 13, 11]
[16, 15, 16, 12, 11, 13]
[19, 13, 17, 9, 15, 14]
[20, 21, 19, 17, 17, 14]
